In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import keras_tuner as kt
from sklearn.decomposition import PCA
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt

In [2]:
# Function for Histogram Equalization
def histogram_equalization(img):
    return cv2.equalizeHist(img)

In [3]:
# Function for Gaussian Blur
def gaussian_blur(img, kernel_size=(5, 5)):
    return cv2.GaussianBlur(img, kernel_size, 0)

In [4]:
# Function for Canny Edge Detection
def canny_edge_detection(img, low_threshold=100, high_threshold=200):
    return cv2.Canny(img, low_threshold, high_threshold)

In [5]:
# Function to preprocess the image with all techniques
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply Histogram Equalization
    img_eq = histogram_equalization(img)
    
    # Apply Gaussian Blur
    img_blur = gaussian_blur(img_eq)
    
    # Apply Canny Edge Detection
    img_edges = canny_edge_detection(img_blur)
    
    return img_edges

In [6]:
# Function to apply PCA after preprocessing
def apply_pca(images, n_components=100):
    pca = PCA(n_components=n_components)
    flat_images = images.reshape(images.shape[0], -1)  # Flatten the images for PCA
    pca_images = pca.fit_transform(flat_images)
    print(f"Original dimensions: {flat_images.shape}, Reduced dimensions: {pca_images.shape}")
    return pca_images


In [7]:
# Load and preprocess images
def load_and_preprocess_images(image_paths):
    images = []
    labels = []
    for path in image_paths:
        img = preprocess_image(path)
        img_resized = cv2.resize(img, (48, 48))  # Resize to 48x48 as required by the model
        images.append(img_resized)
        label = path.split(os.sep)[-2]  # Assuming label is part of the path
        labels.append(label)
    images = np.array(images)
    labels = pd.get_dummies(labels).values  # One-hot encode labels
    return images, labels

In [8]:
# Load image paths
train_image_paths = glob.glob(os.path.join("dataset2", "train", "**", "*.jpg"), recursive=True)
test_image_paths = glob.glob(os.path.join("dataset2", "test", "**", "*.jpg"), recursive=True)


In [9]:
# Preprocess images
train_images, train_labels = load_and_preprocess_images(train_image_paths)
test_images, test_labels = load_and_preprocess_images(test_image_paths)

In [10]:
# Apply PCA for dimensionality reduction
train_images_pca = apply_pca(train_images, n_components=100)
test_images_pca = apply_pca(test_images, n_components=100)

Original dimensions: (28709, 2304), Reduced dimensions: (28709, 100)
Original dimensions: (7178, 2304), Reduced dimensions: (7178, 100)


In [11]:
# Reshape images back to (batch, height, width, channels) format for CNN
train_images_pca = train_images_pca.reshape(-1, 10, 10, 1)  # Reshape to (batch_size, height, width, channels)
test_images_pca = test_images_pca.reshape(-1, 10, 10, 1)


In [12]:
# Image Augmentation with ImageDataGenerator
image_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],  # Adjust brightness
    rescale=1/255.0  # Rescale pixel values to [0, 1]
)

In [13]:
# Augment and fit the model
train_generator = image_generator.flow(train_images_pca, train_labels, batch_size=64)
test_generator = image_generator.flow(test_images_pca, test_labels, batch_size=64)

In [14]:
# Define model-building function for hyperparameter tuning
def build_model(hp):
    model = Sequential()

    # Tune the number of filters for each layer
    model.add(Conv2D(hp.Int('conv1_filters', min_value=32, max_value=128, step=32), (3, 3), padding='same', input_shape=(10, 10, 1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Float('conv1_dropout', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Conv2D(hp.Int('conv2_filters', min_value=64, max_value=256, step=64), (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Float('conv2_dropout', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Flatten())

    # Tune the number of neurons in dense layers
    model.add(Dense(hp.Int('dense_units', min_value=128, max_value=512, step=128), activation='relu'))
    model.add(Dropout(hp.Float('dense_dropout', min_value=0.2, max_value=0.5, step=0.1)))

    # Output layer
    model.add(Dense(7, activation='softmax'))

    # Tune the learning rate for the optimizer
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']),
                  loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [15]:
# Hyperparameter tuning with Keras Tuner
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=20,
                        executions_per_trial=1,
                        directory='hyperparam_tuning_code_final',
                        project_name='emotion_classification_with_hyperparameter_final')

# Early stopping and learning rate reduction callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
model_checkpoint = ModelCheckpoint('best_model_final_run_with_hp.keras', monitor='val_accuracy', save_best_only=True)

# Search for best hyperparameters
tuner.search(train_generator,
             validation_data=test_generator,
             epochs=50,
             callbacks=[early_stopping, reduce_lr, model_checkpoint])

Trial 20 Complete [00h 01m 24s]
val_accuracy: 0.25410977005958557

Best val_accuracy So Far: 0.2566174566745758
Total elapsed time: 00h 32m 08s
